## 즐겨찾는 주유소 14point 표시하기

In [1]:
import time
import pandas as pd
import numpy as np
import folium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse, quote


In [2]:
df = pd.read_excel('./station_data/oil14point.xlsx')
df

,상호,주소,상표,셀프여부,경유
0,㈜늘품오일 한길셀프주유소,인천 서구 검단로 861(불로동),알뜰주유소,Y,1075
1,이마트24서비스 김포본점주유소,경기 김포시 양촌읍 김포대로 1581,SK에너지,N,1099
2,서김포IC주유소,경기 김포시 양촌면 김포대로 1634,GS칼텍스,N,1095
3,대성석유㈜통진대성주유소,경기 김포시 통진읍 하성로 58,GS칼텍스,N,1085
4,카닥일산주유소,경기 고양시 일산동구 백마로 54 (장항1동),GS칼텍스,N,1079
5,신화상사㈜구도일주유소신화일산,경기 고양시 일산동구 백마로 36 (장항동),S-OIL,N,1079
6,씨앤씨네트웍스 직영 제7주유소,경기 수원시 권선구 권선로 644,GS칼텍스,Y,1095
7,수원주유소,경기 수원시 팔달구 인계로 76 (인계동,S-OIL,Y,1095
8,뉴금강주유소,경기 수원시 권선구 매송고색로 577,현대오일뱅크,N,1068
9,지에스칼텍스㈜행정타운주유소,경기 수원시 권선구 권선로 98,GS칼텍스,N,1096


In [3]:
df.info()  # 경유부분 인트64 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상호      14 non-null     object
 1   주소      14 non-null     object
 2   상표      14 non-null     object
 3   셀프여부    14 non-null     object
 4   경유      14 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 688.0+ bytes


In [4]:
stations = pd.DataFrame({
    '상호': df['상호'],
    '주소': df['주소'],
    '가격': df['경유'],
    '셀프': df['셀프여부'],
    '상표': df['상표']
})
stations

,상호,주소,가격,셀프,상표
0,㈜늘품오일 한길셀프주유소,인천 서구 검단로 861(불로동),1075,Y,알뜰주유소
1,이마트24서비스 김포본점주유소,경기 김포시 양촌읍 김포대로 1581,1099,N,SK에너지
2,서김포IC주유소,경기 김포시 양촌면 김포대로 1634,1095,N,GS칼텍스
3,대성석유㈜통진대성주유소,경기 김포시 통진읍 하성로 58,1085,N,GS칼텍스
4,카닥일산주유소,경기 고양시 일산동구 백마로 54 (장항1동),1079,N,GS칼텍스
5,신화상사㈜구도일주유소신화일산,경기 고양시 일산동구 백마로 36 (장항동),1079,N,S-OIL
6,씨앤씨네트웍스 직영 제7주유소,경기 수원시 권선구 권선로 644,1095,Y,GS칼텍스
7,수원주유소,경기 수원시 팔달구 인계로 76 (인계동,1095,Y,S-OIL
8,뉴금강주유소,경기 수원시 권선구 매송고색로 577,1068,N,현대오일뱅크
9,지에스칼텍스㈜행정타운주유소,경기 수원시 권선구 권선로 98,1096,N,GS칼텍스


In [5]:
stations['시구'] = [addr.split()[1] for addr in stations['주소']]
stations

,상호,주소,가격,셀프,상표,시구
0,㈜늘품오일 한길셀프주유소,인천 서구 검단로 861(불로동),1075,Y,알뜰주유소,서구
1,이마트24서비스 김포본점주유소,경기 김포시 양촌읍 김포대로 1581,1099,N,SK에너지,김포시
2,서김포IC주유소,경기 김포시 양촌면 김포대로 1634,1095,N,GS칼텍스,김포시
3,대성석유㈜통진대성주유소,경기 김포시 통진읍 하성로 58,1085,N,GS칼텍스,김포시
4,카닥일산주유소,경기 고양시 일산동구 백마로 54 (장항1동),1079,N,GS칼텍스,고양시
5,신화상사㈜구도일주유소신화일산,경기 고양시 일산동구 백마로 36 (장항동),1079,N,S-OIL,고양시
6,씨앤씨네트웍스 직영 제7주유소,경기 수원시 권선구 권선로 644,1095,Y,GS칼텍스,수원시
7,수원주유소,경기 수원시 팔달구 인계로 76 (인계동,1095,Y,S-OIL,수원시
8,뉴금강주유소,경기 수원시 권선구 매송고색로 577,1068,N,현대오일뱅크,수원시
9,지에스칼텍스㈜행정타운주유소,경기 수원시 권선구 권선로 98,1096,N,GS칼텍스,수원시


In [6]:
stations['시구'].unique() 

array(['서구', '김포시', '고양시', '수원시', '강서구'], dtype=object)

In [7]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상호      14 non-null     object
 1   주소      14 non-null     object
 2   가격      14 non-null     int64 
 3   셀프      14 non-null     object
 4   상표      14 non-null     object
 5   시구      14 non-null     object
dtypes: int64(1), object(5)
memory usage: 800.0+ bytes


In [8]:
gu_data = pd.pivot_table(stations, index=['시구'], values=['가격'], aggfunc=np.mean)
gu_data

,가격
시구,
강서구,1180.000000
고양시,1079.000000
김포시,1093.000000
서구,1056.333333
수원시,1088.500000


In [9]:
#위와같은결과 그룹바이
gu_data2 = stations['가격'].groupby(stations['시구']).mean()
gu_data2

시구
강서구    1180.000000
고양시    1079.000000
김포시    1093.000000
서구     1056.333333
수원시    1088.500000
Name: 가격, dtype: float64

In [10]:
stations.to_csv('stations14point.csv', sep=',', encoding='utf8', index=False)

In [11]:
#도로명주소로 부터 위도, 경도 정보구하기
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [12]:
addr = '검단로 861'
url ='https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(addr)
result = requests.get(url, headers ={"Authorization" : "KakaoAK "+kmap_key}).json()
result

{'documents': [{'address': {'address_name': '인천 서구 불로동 185-1',
    'b_code': '2826012100',
    'h_code': '2826069000',
    'main_address_no': '185',
    'mountain_yn': 'N',
    'region_1depth_name': '인천',
    'region_2depth_name': '서구',
    'region_3depth_h_name': '불로대곡동',
    'region_3depth_name': '불로동',
    'sub_address_no': '1',
    'x': '126.692894954824',
    'y': '37.6195764672622'},
   'address_name': '인천 서구 검단로 861',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '인천 서구 검단로 861',
    'building_name': '',
    'main_building_no': '861',
    'region_1depth_name': '인천',
    'region_2depth_name': '서구',
    'region_3depth_name': '불로동',
    'road_name': '검단로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.692894954824',
    'y': '37.6195764672622',
    'zone_no': '22621'},
   'x': '126.692894954824',
   'y': '37.6195764672622'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [13]:
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])
lat, lng

(37.6195764672622, 126.692894954824)

In [14]:
# station14point.csv파일 읽어서 위도, 경도 정보 추가하기
df = pd.read_csv('stations14point.csv')
df

,상호,주소,가격,셀프,상표,시구
0,㈜늘품오일 한길셀프주유소,인천 서구 검단로 861(불로동),1075,Y,알뜰주유소,서구
1,이마트24서비스 김포본점주유소,경기 김포시 양촌읍 김포대로 1581,1099,N,SK에너지,김포시
2,서김포IC주유소,경기 김포시 양촌면 김포대로 1634,1095,N,GS칼텍스,김포시
3,대성석유㈜통진대성주유소,경기 김포시 통진읍 하성로 58,1085,N,GS칼텍스,김포시
4,카닥일산주유소,경기 고양시 일산동구 백마로 54 (장항1동),1079,N,GS칼텍스,고양시
5,신화상사㈜구도일주유소신화일산,경기 고양시 일산동구 백마로 36 (장항동),1079,N,S-OIL,고양시
6,씨앤씨네트웍스 직영 제7주유소,경기 수원시 권선구 권선로 644,1095,Y,GS칼텍스,수원시
7,수원주유소,경기 수원시 팔달구 인계로 76 (인계동,1095,Y,S-OIL,수원시
8,뉴금강주유소,경기 수원시 권선구 매송고색로 577,1068,N,현대오일뱅크,수원시
9,지에스칼텍스㈜행정타운주유소,경기 수원시 권선구 권선로 98,1096,N,GS칼텍스,수원시


In [15]:
df.index

RangeIndex(start=0, stop=14, step=1)

In [16]:
lat_list = []; lng_list= []
for i in df.index:
    url ='https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(df['주소'][i]) #여기수정

    result = requests.get(url, headers ={"Authorization" : "KakaoAK "+kmap_key}).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
    lat_list.append(lat)
    lng_list.append(lng)

In [17]:
df['lat'] = lat_list
df['lng'] = lng_list
df

,상호,주소,가격,셀프,상표,시구,lat,lng
0,㈜늘품오일 한길셀프주유소,인천 서구 검단로 861(불로동),1075,Y,알뜰주유소,서구,37.619576,126.692895
1,이마트24서비스 김포본점주유소,경기 김포시 양촌읍 김포대로 1581,1099,N,SK에너지,김포시,37.662287,126.661744
2,서김포IC주유소,경기 김포시 양촌면 김포대로 1634,1095,N,GS칼텍스,김포시,37.664051,126.656285
3,대성석유㈜통진대성주유소,경기 김포시 통진읍 하성로 58,1085,N,GS칼텍스,김포시,37.680964,126.632497
4,카닥일산주유소,경기 고양시 일산동구 백마로 54 (장항1동),1079,N,GS칼텍스,고양시,37.643856,126.761811
5,신화상사㈜구도일주유소신화일산,경기 고양시 일산동구 백마로 36 (장항동),1079,N,S-OIL,고양시,37.642758,126.760687
6,씨앤씨네트웍스 직영 제7주유소,경기 수원시 권선구 권선로 644,1095,Y,GS칼텍스,수원시,37.260277,127.021671
7,수원주유소,경기 수원시 팔달구 인계로 76 (인계동,1095,Y,S-OIL,수원시,37.268483,127.025965
8,뉴금강주유소,경기 수원시 권선구 매송고색로 577,1068,N,현대오일뱅크,수원시,37.247586,126.967818
9,지에스칼텍스㈜행정타운주유소,경기 수원시 권선구 권선로 98,1096,N,GS칼텍스,수원시,37.260968,126.963157


In [18]:
df.sort_values(by='가격', ascending=True)
# 인천 서구가 가장 저렴하다! 그 외 수원에도 최저가 있음! 
# 서울중에서도 가장 저렴한 강서구는 이중 비싼편이다... 서울은 서울인가 봄 ㅋ

,상호,주소,가격,셀프,상표,시구,lat,lng
13,강남주유소,인천 서구 봉수대로 184,1037,N,SK에너지,서구,37.492083,126.661449
12,신현주유소,인천 서구 봉수대로501번길2 (신현동),1057,Y,S-OIL,서구,37.520832,126.662082
8,뉴금강주유소,경기 수원시 권선구 매송고색로 577,1068,N,현대오일뱅크,수원시,37.247586,126.967818
0,㈜늘품오일 한길셀프주유소,인천 서구 검단로 861(불로동),1075,Y,알뜰주유소,서구,37.619576,126.692895
4,카닥일산주유소,경기 고양시 일산동구 백마로 54 (장항1동),1079,N,GS칼텍스,고양시,37.643856,126.761811
5,신화상사㈜구도일주유소신화일산,경기 고양시 일산동구 백마로 36 (장항동),1079,N,S-OIL,고양시,37.642758,126.760687
3,대성석유㈜통진대성주유소,경기 김포시 통진읍 하성로 58,1085,N,GS칼텍스,김포시,37.680964,126.632497
2,서김포IC주유소,경기 김포시 양촌면 김포대로 1634,1095,N,GS칼텍스,김포시,37.664051,126.656285
6,씨앤씨네트웍스 직영 제7주유소,경기 수원시 권선구 권선로 644,1095,Y,GS칼텍스,수원시,37.260277,127.021671
7,수원주유소,경기 수원시 팔달구 인계로 76 (인계동,1095,Y,S-OIL,수원시,37.268483,127.025965


In [19]:
df.to_csv('stations14point_latlng.csv', sep=',', encoding='utf8', index=False)

In [20]:
# 써클로 위치만 표시 하기
map = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=10.5)

for n in range(len(df)) :
    folium.CircleMarker([df['lat'][n], df['lng'][n]],radius=15, color = '#cd3181', fill_color = '#CD3181').add_to(map)
map

In [21]:
# 써클에 툴팁으로 주유소정보표시하기 
# pip install git+https://github.com/python-visualization/branca.git@master 설치하고 vscode재접속, 재부팅했는데도 왜 깨져나올까요 ㅠㅠㅠ
map = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=10.5)

for n in range(len(df)) :
    folium.CircleMarker([df['lat'][n], df['lng'][n]],radius=15, color = '#cd3181', fill_color = '#CD3181',
    popup=df['상호'][n], tooltip=f'{df["가격"][n]}' ).add_to(map)
map

In [22]:
# tooltip표시/ 가장저렴한주유소 Red
# 이상한데,  왜 에러안나지? 06_marker수정중............
def marker():
    df = pd.read_csv('stations14point_latlng.csv')
    df['icon'] = ['info-sign']
    palette = ['gray', 'red']
    mapping = folium.Map(location=[df.lat.mean(), df.lng.mean()],
                    zoom_start=12)
    for i in df.index:
        folium.Marker(
            location=[df.lat[i], df.lng[i]],
            popup = df.주소[i],
            tooltip = df.가격[i],
            icon=folium.Icon(color=palette[df.weather[i]], icon=df.icon)
        ).add_to(mapping)
    return mapping._repr_html_()

In [23]:
# 가장 저렴한 주유소는 red컬러,heart아이콘
# 아이콘(glyphicons)모양은 https://getbootstrap.com/docs/3.3/components/ 폴리움부트스트랩
map = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=10.5)

for i in df.index:
    frame = f"{df['상호'][i]}"
    popup = folium.Popup(frame, max_width=130)
    if df['가격'][i] == min(df['가격']):
        folium.Marker(
            location = [df['lat'][i], df['lng'][i]],
            tooltip=(f"경유:{df['가격'][i]}원, 주소:{df['주소'][i]}"),
            popup=popup,
            icon=folium.Icon(color='red', icon='heart')
            ).add_to(map)
    else:
        folium.Marker(
            location = [df['lat'][i], df['lng'][i]],
            tooltip=(f"경유:{df['가격'][i]}원, 주소:{df['주소'][i]}"),
            popup=popup,
            icon=folium.Icon(color='blue', icon='flag')
            ).add_to(map)
map

In [24]:
# 가장 저렴한 주유소는 핑크컬러,써클로 표시하기
map = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=10.5)

for i in df.index:
    frame = f"{df['상호'][i]}"
    popup = folium.Popup(frame, max_width=130)
    if df['가격'][i] == min(df['가격']):
        folium.CircleMarker([df['lat'][i], df['lng'][i]], radius=15, color = '#cd3181', fill_color = '#CD3181',
            tooltip=(f"경유:{df['가격'][i]}원, 주소:{df['주소'][i]}"),
            popup=popup,
            ).add_to(map)
    if df['가격'][i] == max(df['가격']):
        folium.CircleMarker([df['lat'][i], df['lng'][i]], radius=15, color = '#7f7f7f', fill_color = '#7f7f7f',
            tooltip=(f"경유:{df['가격'][i]}원, 주소:{df['주소'][i]}"),
            popup=popup,
            ).add_to(map)        
    else:
        folium.CircleMarker([df['lat'][i], df['lng'][i]], radius=15, color = '#4B89DC', fill_color = '#4B89DC',
            tooltip=(f"경유:{df['가격'][i]}원, 주소:{df['주소'][i]}"),
            popup=popup,
            ).add_to(map)
map